In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

from numpy import pi as π

import perceval as pv
from scipy.optimize import minimize
from perceval.components import *

from collections import Counter

from submission.circuits import fullCircuit
import matplotlib.pyplot as plt

import csv

In [2]:
# Fraction of [1, 0, x, y] states
def getFidelityRho1(counts):
    totCounts, validCounts = 0, 0

    for state in counts:
        totCounts += counts[state]
        if state[0] == 1 and state[1] == 0:
            validCounts += counts[state]

    return validCounts/totCounts


# Fraction of [x, y, 1, 0] states
def getFidelityRho2(counts):
    totCounts, validCounts = 0, 0

    for state in counts:
        totCounts += counts[state]
        if state[2] == 1 and state[3] == 0:
            validCounts += counts[state]

    return validCounts/totCounts

def getFidelities(counts):
    return (getFidelityRho1(counts), getFidelityRho2(counts))

def costFunctionAtPhi(counts):
    return (1 - getFidelityRho1(counts))**2 + (1 - getFidelityRho2(counts))**2 + (getFidelityRho1(counts)-getFidelityRho2(counts))**2

In [3]:
def computeFidelity(x0, testPhi = π/2, brightness = 0):

    circuit = fullCircuit(evolTheta=x0, prepTheta=π/2, prepPhi=testPhi)

    p = pv.Processor("SLOS", circuit, noise=pv.NoiseModel(brightness=brightness)) if brightness > 0 else pv.Processor("SLOS", circuit)
    p.with_input(pv.BasicState([0, 1, 0, 1]))

    # The sampler holds 'probs', 'sample_count' and 'samples' calls. You can use the one that fits your needs!
    sampler = pv.algorithm.Sampler(p)  
    sample_count = sampler.sample_count(1000)['results']
    
    valid_counts = {}
    for c in sample_count:
        if c[0] != c[1] and c[2] != c[3]: #Since we introduce only 2 photons in the circuit it is enough
            valid_counts[c] = sample_count[c]

    # print(valid_counts)
    fr1 = getFidelityRho1(valid_counts)
    fr2 = getFidelityRho2(valid_counts)

    # print(fr1)
    # print(fr2)

    return fr1, fr2

In [4]:
def loss_function(theta = [0] * 12, brightness=0):
    cost = 0
    Phi = [ 0, π/2, π, 3*π/2 ]

    # Evaluate the 4 cost functions, one for each fidelity
    for phi_i in Phi:
        circuit = fullCircuit(evolTheta=theta, prepTheta=π/2, prepPhi=phi_i)
        p = pv.Processor("SLOS", circuit, noise=pv.NoiseModel(brightness=brightness)) if brightness > 0 else pv.Processor("SLOS", circuit)
        # p.min_detected_photon_filter(0)
        p.with_input(pv.BasicState([0, 1, 0, 1]))

        # The sampler holds 'probs', 'sample_count' and 'samples' calls. You can use the one that fits your needs!
        sampler = pv.algorithm.Sampler(p)  
        sample_count = sampler.sample_count(1000)['results']

        valid_counts = {}
        for c in sample_count:
            if c[0] != c[1] and c[2] != c[3]: #Since we introduce only 2 photons in the circuit it is enough
                valid_counts[c] = sample_count[c]
        
        cost += costFunctionAtPhi(valid_counts)
        
    return cost

In [5]:
def optimCallback(intermediate_result):
    
    outLine = f'{intermediate_result.fun:0.3f}, '
    for x in intermediate_result.x:
        outLine += f'{x}, '
    outLine += '\n'
    
    global outFileName
    print(outLine)
    with open(outFileName, 'a+') as outFile:
        outFile.writelines([outLine])

In [6]:
class ao():
    def __init__(self, x0):
        self.x = x0

def minimizeVal(swapMimimizers, x0 = None, maxIter = 100, brightness = 0):
    res = ao((np.random.rand(1, 12)[0])*2*π) if x0 == None else ao(x0)

    print(res.x)

    global outFileName
    with open(outFileName, 'w') as outFile:
        outFile.writelines('Fun, p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11')

    for minimizerSwapIndex in range(swapMimimizers):
        res = minimize(loss_function,
            res.x,
            method="Nelder-Mead",
            bounds=[(0, 2*π)]*12,
            callback=optimCallback,
            options={'maxiter':maxIter},
            args=(brightness)
        )
        optimCallback(res)


        res = minimize(loss_function,
            res.x,
            method="powell",
            bounds=[(0, 2*π)]*12,
            callback=optimCallback,
            args=(brightness)
        )
        optimCallback(res)


    return res

In [ ]:
for b in [0.1, 0.3, 0.5, 0.75, 1]:
    global outFileName
    outFileName=f'ResultFile_noisy_{b}.csv'
    res = minimizeVal(5, brightness=b)
    computeFidelity(res.x)